# **Mount the Google Drive**



# **Install packages**

In [2]:
!pip install tensorflow-gpu==2.4.0 tensorboard==2.4.0 tensorboard-plugin-profile==2.4.0

     |████████████████████████████████| 394.7 MB 19 kB/s 
     |████████████████████████████████| 10.6 MB 34.8 MB/s 
     |████████████████████████████████| 1.2 MB 29.7 MB/s 
     |████████████████████████████████| 2.9 MB 32.0 MB/s 
     |████████████████████████████████| 462 kB 44.9 MB/s 
     |████████████████████████████████| 3.8 MB 38.4 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.39.0
    Uninstalling grpcio-1.39.0:
      Successfully uninstalled grpcio-1.39.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successf

In [3]:
import os
os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'

# **GPU available**

In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16256147297840730029, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11345264640
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12313724059573670012
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from datetime import datetime

import numpy as np
import tensorflow as tf

from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

import itertools
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import pandas as pd

import tensorflow.keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [ ]:
tensorflow.keras.__version__

'2.6.0'

In [ ]:
SEED = 0
np.random.seed(SEED)

In [ ]:
RAIN_RATE = 2.0 # adjust rate between the two type, rainy and non-rainy
N_PREDICTOR = 15 #adjust predictor numbers

In [ ]:
CLOUD_TRAIN_FOLDER = '/content/drive/MyDrive/CloudClassificationGPU/train_10mm/'
CLOUD_FLORENCE_FOLDER = '/content/drive/MyDrive/CloudClassificationGPU/florence_10mm/'
CLOUD_TEST_FOLDER = '/content/drive/MyDrive/CloudClassificationGPU/test_10mm/'

TIME = 'day'
RAIN_CLOUD_FILENAME = TIME + '_rain_imerg.txt'
NORAIN_CLOUD_FILENAME = TIME + '_norain_imerg.txt'

'day_rain_imerg.txt'

In [ ]:
def load_data(folder, ratio):
  rain_cloud_path = folder + RAIN_CLOUD_FILENAME
  norain_cloud_path = folder + NORAIN_CLOUD_FILENAME
  print(rain_cloud_path)

  rain_cloud_array = np.loadtxt(rain_cloud_path, delimiter=' ', dtype=np.float32, skiprows=0)
  norain_cloud_array = np.loadtxt(norain_cloud_path, delimiter=' ', dtype=np.float32, skiprows=0)

  rain_cloud_array = rain_cloud_array[~np.isnan(rain_cloud_array).any(axis=1)]
  norain_cloud_array = norain_cloud_array[~np.isnan(norain_cloud_array).any(axis=1)]

  rain_size = rain_cloud_array.shape[0]
  norain_size = norain_cloud_array.shape[0]

  smaller_size = min(rain_size, norain_size)
  new_rain_size = int(smaller_size // RAIN_RATE)
  new_norain_size = smaller_size

  random_rain_indices = np.random.choice(rain_size, size=new_rain_size, replace=False)
  random_norain_indices = np.random.choice(norain_size, size=new_norain_size, replace=False)
  new_rain_cloud_array = rain_cloud_array[random_rain_indices, :]
  new_norain_cloud_array = norain_cloud_array[random_norain_indices, :]

  cloud_array = np.concatenate((new_rain_cloud_array, new_norain_cloud_array), axis=0)
  cloud_array = shuffle(cloud_array)
  return cloud_array


In [ ]:

cloud_train = load_data(CLOUD_TRAIN_FOLDER, ratio=RAIN_RATE)
cloud_florence = load_data(CLOUD_FLORENCE_FOLDER, ratio=RAIN_RATE)

cloud_train_all = np.concatenate((cloud_train, cloud_florence), axis=0)
cloud_train_all = np.repeat(cloud_train_all, repeats=6, axis=0)
cloud_test = load_data(CLOUD_TEST_FOLDER, ratio=RAIN_RATE)

/content/drive/MyDrive/CloudClassificationGPU/train_10mm/day_rain_imerg.txt
/content/drive/MyDrive/CloudClassificationGPU/florence_10mm/day_rain_imerg.txt
/content/drive/MyDrive/CloudClassificationGPU/test_10mm/day_rain_imerg.txt


In [ ]:
def split_data(cloud_arr):
  x = cloud_arr[:, :N_PREDICTOR]
  labels = cloud_arr[:, -1]
  y = to_categorical(labels)
  return x, y, labels

x_org, y_org, labels_org = split_data(cloud_train_all)
x_test, y_test, labels_test = split_data(cloud_test)

In [ ]:
scaler = MinMaxScaler()

x_org = scaler.fit_transform(x_org)
x_test = scaler.transform(x_test)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    x_org, 
    y_org,
    test_size=0.2, 
    random_state=SEED
)

In [ ]:
len(x_train)

2206718

In [ ]:
INPUT_DIM = N_PREDICTOR

def build_classifier():
  model = Sequential()
  model.add(Dense(units=512, input_dim=INPUT_DIM, kernel_initializer='normal', activation='relu'))
  model.add(Dense(units=256, kernel_initializer='normal', activation='relu'))
  model.add(Dense(units=128, kernel_initializer='normal', activation='tanh'))
  model.add(Dense(units=2, kernel_initializer='normal', activation='softmax'))
  
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

estimator = KerasClassifier(build_fn=build_classifier)

In [ ]:
LOGS = '/content/gdrive/MyDrive/CloudClassificationGPU/logs/' + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir = LOGS,
    histogram_freq = 1,
    profile_batch = (2, 8)
)

EPOCHS = 100

history = estimator.fit(
    x_train,
    y_train,
    epochs=EPOCHS,
    batch_size=128,
    validation_data=(x_val, y_val), 
    verbose=True,
    callbacks = [tboard_callback]
)

Epoch 1/100
17240/17240 [==============================] - 69s 4ms/step - loss: 0.3341 - accuracy: 0.8525 - val_loss: 0.3253 - val_accuracy: 0.8549
Epoch 2/100
17240/17240 [==============================] - 65s 4ms/step - loss: 0.3208 - accuracy: 0.8566 - val_loss: 0.3154 - val_accuracy: 0.8584
Epoch 3/100
17240/17240 [==============================] - 64s 4ms/step - loss: 0.3134 - accuracy: 0.8595 - val_loss: 0.3151 - val_accuracy: 0.8599
Epoch 4/100
17240/17240 [==============================] - 65s 4ms/step - loss: 0.3081 - accuracy: 0.8618 - val_loss: 0.3133 - val_accuracy: 0.8589
Epoch 5/100
17240/17240 [==============================] - 64s 4ms/step - loss: 0.3044 - accuracy: 0.8634 - val_loss: 0.3013 - val_accuracy: 0.8646
Epoch 6/100
17240/17240 [==============================] - 68s 4ms/step - loss: 0.3009 - accuracy: 0.8649 - val_loss: 0.3001 - val_accuracy: 0.8651
Epoch 7/100
17240/17240 [==============================] - 65s 4ms/step - loss: 0.2979 - accuracy: 0.8663 - val_